# Test model 1, input your file you want to load

## Imports

In [ ]:
import os
from glob import glob
from math import floor
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, Input, UpSampling2D
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, Flatten, Dense, Reshape, MaxPooling2D,  MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model
from tensorflow.keras.backend import int_shape
from tensorflow.keras.models import load_model

In [ ]:
#SET UP HERE CHANGE THE VALUE TO YOUR FILE DIRECTORY
os.chdir('C:/Users/YOUR_USER/Documents/Colourising_Photos/')
height, width = (256,256)
#Setting up path for data
inputPath = os.getcwd()+"/example_inputs/p_256/*.jpg"
outputPath = (os.getcwd()+"/Test_output/Model1/")

## Create the model to load our weights into

In [ ]:
inputs = Input(name = 'input',shape = (256,256,1))
x = inputs
index = 0

layer_filters = [32,64,128];

for current in layer_filters:
    x = Conv2D(filters = current, kernel_size = 3, activation='relu', padding='same', strides= 2)(x)
    x = Dropout(0.2)(x)

shape = int_shape(x) 
x = Flatten()(x)
x = Dense(256)(x)

x = Dense(shape[1]*shape[2]*shape[3])(x)
x = Reshape((shape[1],shape[2],shape[3]))(x)

index = 2
for current in layer_filters[::-1]:
    x = Conv2DTranspose(filters = current, kernel_size = 3,  activation='relu', padding="same", strides=2)(x)
    x = Dropout(0.2)(x)
    index += -1

outputs = Conv2DTranspose(filters=3, kernel_size = 3, activation='sigmoid',padding='same')(x)


model = Model(inputs, outputs, name='model')
model.load_weights(""+os.getcwd()+"/Model1/model1_1000.h5")

model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

### Load in data we want to colourise, using previously defined path

In [ ]:
colourise = []

pics = glob(inputPath)
index = 0;

for p in pics:
    #Take photos by value
    picName = pics[index].split('\\')[1].split('.')[0]
    img = Image.open(pics[index])
    index += 1
    
    #in case photo passed is not B&W
    imgBW = img.convert('L')  

    BW = np.array(imgBW).astype('float32')
    BW /= 255
    colourise.append(BW)
    img.close()

colourise = np.asarray(colourise).flatten()
colourise = colourise.reshape(-1,width,height,1)


### Predict and then save to /test_output/

In [ ]:
# Predict
output = model.predict(colourise[:])
#Now return the values from 0->1 to 0->255
output *= 255
#Convert from f_32 -> u_int8
output = output.astype('uint8')
#save train images
photo_num = 1
for img in output:
    output = Image.fromarray(img)
    output.save(outputPath+'ouput{}.png'.format(photo_num))
    photo_num += 1